In [38]:
import json
import lzma
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from dotenv import load_dotenv 
import ast
import random
import numpy as np
import json

In [39]:
# loading variables from .env file
load_dotenv("../../private_data/.env") 

# PARENT gets us to the root of the project
PARENT = "./../../"

FOLDER_TABLE = PARENT + os.getenv("FOLDER_TABLE")
FILE_FABRITIUS_DATA = PARENT + os.getenv("FILE_FABRITIUS_DATA")
FILE_FABRITIUS_DATA_FILTERED = PARENT + os.getenv("FILE_FABRITIUS_DATA_FILTERED")
FILE_FABRITIUS_DATA_FILTERED_DOWNLOADED = PARENT + os.getenv("FILE_FABRITIUS_DATA_FILTERED_DOWNLOADED")
FOLDER_FIGURES = PARENT + os.getenv("FOLDER_FIGURES")
IMAGES_FOLDER = PARENT + os.getenv("IMAGES_FOLDER")
RECORD_IDS_TESTING_SET = PARENT + os.getenv("RECORD_IDS_TESTING_SET")
RECORD_IDS_VALIDATION_SET = PARENT + os.getenv("RECORD_IDS_VALIDATION_SET")
FILE_FABRITIUS_ICONOGRAPHIES_JSON = PARENT + os.getenv("FILE_FABRITIUS_ICONOGRAPHIES_JSON")

In [3]:
filtered_data_downloaded = pd.read_csv(FILE_FABRITIUS_DATA_FILTERED_DOWNLOADED)
filtered_data_downloaded = filtered_data_downloaded[filtered_data_downloaded["recordID"] != 11546]
filtered_data_downloaded = filtered_data_downloaded[filtered_data_downloaded["recordID"] != 5262]
filtered_data_downloaded

,recordLanguage,recordID,DatabaseId,LinkToVubis,objectWork.creatorDescription,objectWork.termClassification,objectWork.workID,objectWork.titleText,objectWork.objectWorkType,objectWork.measurementsDescription,...,objectWorkType_is_esquisse,objectWorkType_is_estampe,objectWorkType_is_grisaille,objectWorkType_is_photographie,objectWorkType_is_polyptyque,objectWorkType_is_retable,objectWorkType_is_sanguine,objectWorkType_is_triptyque,objectWorkType_is_ébauche,contains_desired_objectWorkType
0,fre,10002,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,Henry de Groux,peinture (Dept. Art Moderne),12315,Portrait de Paul Verlaine,['tableau (toile)'],"[46.0, 38.0]",...,False,False,False,False,False,False,False,False,False,True
1,fre,10004,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,Pierre Oyens,peinture (Dept. Art Moderne),3230,Les collègues,['tableau (toile)'],"[202.0, 177.0]",...,False,False,False,False,False,False,False,False,False,True
2,fre,10005,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,Ferdinand Roybet,peinture (Dept. Art Moderne),6831,Portrait de femme,['tableau (panneau)'],"[81.0, 54.0]",...,False,False,False,False,False,False,False,False,False,True
3,fre,10016,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,Henry de Groux,peinture (Dept. Art Moderne),12502,Christ aux outrages,['tableau (toile)'],"[173.0, 145.0]",...,False,False,False,False,False,False,False,False,False,True
4,fre,1003,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,Alfred Stevens,peinture (Dept. Art Moderne),10714,Dame au chien,['tableau (panneau)'],"[26.9, 20.9]",...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3007,fre,9959,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,"Eugène Laermans, baron",oeuvre sur papier (Dept. Art Moderne),12312,"Femme, vue de dos",['dessin'],NaN,...,False,False,False,False,False,False,False,False,False,True
3008,fre,9960,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,"Eugène Laermans, baron",oeuvre sur papier (Dept. Art Moderne),12311,Sur le champ,['dessin'],NaN,...,False,False,False,False,False,False,False,False,False,True
3009,fre,9962,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,Fernand Khnopff,oeuvre sur papier (Dept. Art Moderne),12264,Une aile bleue,['photographie'],NaN,...,False,False,False,True,False,False,False,False,False,True
3010,fre,999,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,Constant Montald,peinture (Dept. Art Moderne),10613,La baignade,['détrempe'],"[525.0, 593.0]",...,False,False,False,False,False,False,False,False,False,True


In [21]:
filtered_data_downloaded["subjectMatter.subjectTerms"]

0                                  figure (homme ; barbe)
1       groupe de figures (homme : chapeau ; femme : a...
2                   figure (femme : en buste ; de profil)
3       groupe de figures (homme ; femme ; foule ; sol...
4       figure (femme : en pied ; de profil ; vêtement...
                              ...                        
3007    figure (femme : de dos ; en pied ; vêtement : ...
3008    groupe de figures (femme : en pied ; enfant ; ...
3009    figure (femme : voile ; sculpture : tête ; ail...
3010    groupe de figures (femme : nu ; couvre-chef : ...
3011    Groupe de figures (femme ; homme ; enfant : nu...
Name: subjectMatter.subjectTerms, Length: 3010, dtype: object

In [4]:
filtered_data_downloaded["subjectMatter.iconographicTerms"]

0       portrait (Paul Verlaine ; homme de lettres ; p...
1           scène (atelier d'artiste ; artiste : peintre)
2                                                portrait
3       scène biblique ([SO] : Nouveau Testament : Eva...
4                                                     NaN
                              ...                        
3007                                                  NaN
3008                                               paysan
3009                          divinité : Hypnos : sommeil
3010                                     scène (baignade)
3011                           divinité ; nymphe ; satyre
Name: subjectMatter.iconographicTerms, Length: 3010, dtype: object

In [34]:
class Node:
    def __init__(self, value, children=None, parent=None):
        self.value = value#lower()
        while self.value[0]==" ":
            self.value = self.value[1:]
        self.parent = parent
        
        if children is None:
            self.children = []
        else:
            self.children = children
            for child in children:
                child.set_parent(self)

        self.forgotten = False
    
    def setForgotten(self, forgotten):
        self.forgotten = forgotten

    def __hash__(self):
        return hash(self.value)
    
    def __eq__(self, other):
        return self.value == other.value

    def get_as_string(self, level=0):
        st = f"'{self.value}'"
        children_st = ",".join([child.get_as_string(level+1) for child in self.children])

        if self.value.startswith("<depth"):
            return children_st

        if len(self.children) > 0:
            st += f"[{children_st}]"

        return st

    def set_parent(self, parent):
        self.parent = parent

    def get_parent(self):
        return self.parent

    def add_child(self, child):
        self.children.append(child)
        child.set_parent(self)
    
    def print_tree(self, level=0):
        print("\t"*level + self.value)
        for child in self.children:
            child.print_tree(level+1)

    def get_children(self):
        return self.children
    
    def getUniqueLabels(self):
        labels = set()
        labels.add(self.value)
        for child in self.children:
            labels.update(child.getUniqueLabels())
        return labels
    
    def remove_empty_children(self):
        newChildren = []
        for child in self.children:
            if len(child.children) > 0:
                newChildren.append(child)
        self.children = newChildren

    def as_json(self, depth=0):
        return {
            "value": self.value,
            "children": [child.as_json(depth=depth + 1) for child in self.children],
            "depth": depth
        }

In [35]:
delimiters = ["(", ")", ";", ":"]

def cleanIconography(iconography):
    while "  " in iconography:
        iconography = iconography.replace("  ", " ")
    for delimiter in delimiters + ["/"]:
        iconography = iconography.replace(delimiter, f" {delimiter}")
        iconography = iconography.replace(delimiter, f"{delimiter} ")
    return iconography

def l1(elements, parent):
    e1 = elements[0]
    if e1 not in delimiters:
        parent.add_child(Node(e1))
    
def l2(elements, parent):
    # S )
    # ; S
    e1 = elements[0]
    e2 = elements[1]

    rule1 = (e1 not in delimiters) and (e2==")")
    if rule1:
        parent.add_child(Node(e1))
        return
    
    rule2 = (e1==";") and (e2 not in delimiters)
    if rule2:
        parent.add_child(Node(e2))
        return

def explorer(elements, parent):

    L = len(elements)
    if L==0:
        return
    if L==1:
        return l1(elements, parent)
    if L==2:
        return l2(elements, parent)

    e1 = elements[0]
    e2 = elements[1]
    e3 = elements[2]

    # S ; S
    rule1 = (e1 not in delimiters) and (e2==";") and (e3 not in delimiters)
    if rule1:
        parent.add_child(Node(e1))
        return explorer(elements[2:], parent)
    
    # S : S
    rule2 = (e1 not in delimiters) and (e2==":") and (e3 not in delimiters)
    if rule2:
        newNode = Node(e1)
        parent.add_child(newNode)
        newNode.add_child(Node(e3))
        return explorer(elements[3:], parent)
    
    # S ( S
    rule3 = (e1 not in delimiters) and (e2=="(") and (e3 not in delimiters)
    if rule3:
        newNode = Node(e1)
        parent.add_child(newNode)
        return explorer(elements[2:], newNode)

    # S ) E
    rule4 = (e1 not in delimiters) and (e2==")")
    if rule4:
        parent.add_child(Node(e1))
        return explorer(elements[2:], parent.get_parent())
    
    # S ; E
    rule5 = (e1 not in delimiters) and (e2==";")
    if rule5:
        # A litte trick: We add a new node after ';' since we know that E is not a S
        # We add <group> after ;
        parent.add_child(Node(e1))
        return explorer(["<group>"] + elements[2:], parent)

    # ; ( E
    rule6 = (e1==";") and (e2=="(")
    if rule6:
        # A litte trick: We add a new node after ';' to create a new group
        # We add <group> after ;
        return explorer(["<group>"] + elements[1:], parent)

    # ; S )
    rule7 = (e1==";") and (e2 not in delimiters) and (e3==")")
    if rule7:
        parent.add_child(Node(e2))
        return explorer(elements[3:], parent.get_parent())
    
    # ; S E
    rule8 = (e1==";") and (e2 not in delimiters)
    if rule8:
        return explorer(elements[1:], parent)

    # : S E
    rule9 = (e1==":") and (e2 not in delimiters)
    if rule9:
        lastChild = parent.children[-1]
        lastChild.add_child(Node(e2))
        return explorer(elements[2:], parent)

    # ) ; E
    rule10 = (e1==")") and (e2==";")
    if rule10:
        return explorer(elements[2:], parent.get_parent())

    # ( E E
    rule11 = (e1=="(")
    if rule11:
        newNode = Node("<group>")
        parent.add_child(newNode)
        return explorer(elements[1:], newNode)

    # Error correction
    # :/; :/; S
    rule12 = (e1 in ";:") and (e2 in ";:") and (e3 not in delimiters)
    if rule12:
        # We discard the first separator
        return explorer(elements[1:], parent)

    raise Exception(f"Unknown rule: {elements}")

def raw_iconography_to_tree(iconography):
    iconography = cleanIconography(iconography)
    
    # Split by "/"
    iconography = iconography.split("/")
    if len(iconography) > 1:
        # This should not happen !
        iconography = iconography[0]
    
    # Create the root node
    root = Node("root")

    for i, depth in enumerate(iconography):

        iconography_for_depth = iconography[i]

        text = ""
        iconography_for_depth_split = []
        for ch in iconography_for_depth:
            if ch in delimiters:
                iconography_for_depth_split.append(text)
                iconography_for_depth_split.append(ch)
                text = ""
            else:
                text += ch
        iconography_for_depth_split.append(text)

        # Remove empty strings
        iconography_for_depth_split = [x.strip() for x in iconography_for_depth_split]  
        iconography_for_depth_split = [x for x in iconography_for_depth_split if x != ""]
        
        # Correct mistakes
        # 1) Correct ) ( to ) ; (
        corrected = []
        index = 0
        while index < len(iconography_for_depth_split)-1:
            if iconography_for_depth_split[index] == ")" and iconography_for_depth_split[index+1] == "(":
                corrected.append(")")
                corrected.append(";")
                corrected.append("(")
                index += 2
            else:
                corrected.append(iconography_for_depth_split[index])
                index += 1

        # Create the nodes for the depth  
        explorer(corrected, root)

    # Remove <depth> nodes if they don't have ay children
    root.remove_empty_children()

    return root

# Test 
iconography = "scène religieuse ([SO] : Jacques de Voragine : Légende dorée :  ; saint Sébastien : martyre ; ange)/groupe de figures (homme : blessé ; être imaginaire : aile ; arme : épée ; flèche ; cuirasse)"
tree = raw_iconography_to_tree(iconography)
print(iconography)
tree.print_tree()
print(tree.as_json())

scène religieuse ([SO] : Jacques de Voragine : Légende dorée :  ; saint Sébastien : martyre ; ange)/groupe de figures (homme : blessé ; être imaginaire : aile ; arme : épée ; flèche ; cuirasse)
root
{'value': 'root', 'children': [], 'depth': 0}


In [44]:
processed_iconographies = {}
for index, row in filtered_data_downloaded.iterrows():

    recordID = row["recordID"]
    subjectTerms = row["subjectMatter.subjectTerms"]

    tree = raw_iconography_to_tree(subjectTerms)
    tree = json.loads(json.dumps(tree.as_json()))

    processed_iconographies[recordID] = tree

# Save
with open(FILE_FABRITIUS_ICONOGRAPHIES_JSON, "w", encoding="utf-8") as f:
    json.dump(processed_iconographies, f, indent=4, ensure_ascii=False)